In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv("train.csv")
test= pd.read_csv("test.csv")

In [44]:
train['education'] = train['education'].replace('basic.9y', 'basic')
train['education'] = train['education'].replace('basic.6y', 'basic')
train['education'] = train['education'].replace('basic.4y', 'basic')

In [45]:
train.shape, test.shape

((28831, 22), (12357, 7))

In [46]:
test['education'] = test['education'].replace('basic.9y', 'basic')
test['education'] = test['education'].replace('basic.6y', 'basic')
test['education'] = test['education'].replace('basic.4y', 'basic')

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [9]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
col = ['customer_id', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']
for cols in col:
    train[cols] = lb.fit_transform(train[cols])
col = ['customer_id', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']
for cols in col:
    test[cols] = lb.fit_transform(test[cols])
        

In [10]:
#picking X and y
X = train.drop(["customer_id", "subscribed"], axis =1)
y = train["subscribed"]

test = test.drop(["customer_id"], axis = 1)


In [31]:
X.columns

Index(['duration', 'euribor3m', 'cons_conf_idx', 'nr_employed', 'emp_var_rate',
       'pdays', 'poutcome', 'month'],
      dtype='object')

In [32]:
X =  train[['duration', 'euribor3m', 'cons_conf_idx', 'nr_employed', 'emp_var_rate', 'pdays', 'poutcome', 'month']]


In [35]:
test = pd.read_csv('test.csv')

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.feature_selection import VarianceThreshold

In [37]:
test =  test[['duration', 'euribor3m', 'cons_conf_idx', 'nr_employed', 'emp_var_rate', 'pdays', 'poutcome', 'month']]


In [38]:
filter = VarianceThreshold(0.5)
X = filter.fit_transform(X)

test = filter.transform(test)



In [14]:
scaler = StandardScaler()   # MinMaxScaler
X = scaler.fit_transform(X)

test = scaler.transform(test)

In [39]:
import catboost as cat
catm = cat.CatBoostClassifier(random_seed=1996, bootstrap_type='Bernoulli', 
                              iterations=1000, silent=True, eval_metric='Logloss')


In [40]:
%%time
catm.fit(X, y, eval_set=[(X,y)], early_stopping_rounds=350, verbose=200); #stopping from 350 to 300

Learning rate set to 0.072693
0:	learn: 0.5839233	test: 0.5839233	best: 0.5839233 (0)	total: 34.1ms	remaining: 34.1s
200:	learn: 0.1675506	test: 0.1675506	best: 0.1675506 (200)	total: 4.51s	remaining: 17.9s
400:	learn: 0.1587520	test: 0.1587520	best: 0.1587520 (400)	total: 8.58s	remaining: 12.8s
600:	learn: 0.1526968	test: 0.1526968	best: 0.1526968 (600)	total: 12.8s	remaining: 8.48s
800:	learn: 0.1478352	test: 0.1478352	best: 0.1478352 (800)	total: 16.6s	remaining: 4.13s
999:	learn: 0.1438431	test: 0.1438431	best: 0.1438431 (999)	total: 20.6s	remaining: 0us

bestTest = 0.1438430986
bestIteration = 999

Wall time: 25.3 s


In [41]:
cat_pred = catm.predict(test)

CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=6]="may": Cannot convert 'b'may'' to float

In [ ]:
cat_pred.shape

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, plot_confusion_matrix, plot_roc_curve
print("done")

In [ ]:

lgbm = lgb.LGBMClassifier(random_state=1996, n_estimators=1000, metric='binary_logloss', learning_rate=0.04, subsample=0.7,
                         colsample_bytree=0.8, num_leaves=64, reg_alpha=4, min_child_samples=5,subsample_freq=2)

In [ ]:
lgbm.fit(X, y, eval_set=[(X,y), (X,y)], verbose=200, early_stopping_rounds=200);

In [ ]:
lgb_pred = lgbm.predict(test)

In [ ]:
blend = (0.6 * lgb_pred + 0.4)

In [42]:
sample_submission = pd.read_csv("sample_submission.csv")
test_id = sample_submission.customer_id
lgb_df =  pd.DataFrame({'customer_id' : test_id, 'subscribed' : lgb_pred})
cat_df =  pd.DataFrame({'customer_id' : test_id, 'subscribed' : cat_pred})
blend_df =  pd.DataFrame({'customer_id' : test_id, 'subscribed' : blend})

lgb_df.to_csv('lgblast.csv', index=False)
cat_df.to_csv('catlast.csv', index=False)
blend_df.to_csv('blendlast.csv', index=False)

In [43]:
test

array([[6.183333333333334, 2.301, -32.34, ..., -1.77, 999, 'may'],
       [4.75, 5.862, -25.48, ..., 1.13, 999, 'may'],
       [0.8666666666666667, 2.3150000000000004, -32.34, ..., -1.77, 999,
        'may'],
       ...,
       [4.183333333333334, 5.964, -29.89, ..., 1.43, 999, 'jul'],
       [0.8333333333333334, 5.968, -25.27, ..., 1.43, 999, 'aug'],
       [3.683333333333333, 5.122000000000001, -29.4, ..., -0.07, 999,
        'nov']], dtype=object)